<a href="https://colab.research.google.com/github/roydendsouza31/AI-Generated-Images-vs-Real-Images/blob/A1/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI Generated Images vs Real Images

In [14]:
from google.colab import drive
drive.mount('/content/drive')

!pip install kaggle

import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

!kaggle datasets download -d birdy654/cifake-real-and-ai-generated-synthetic-images

import zipfile

# Define the path to your zip file
file_path = '/content/cifake-real-and-ai-generated-synthetic-images.zip'  # Replace 'your_file.zip' with your file's name

!mkdir CIFAKE

# Unzip the file to a specific destination
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/CIFAKE')  # Replace 'destination_folder' with your desired folder

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cifake-real-and-ai-generated-synthetic-images.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘CIFAKE’: File exists


In [15]:
dataset_dir = "/content/CIFAKE" # For Kaggle notebooks. If you run locally, point this line to the CIFAKE directory
print("Loading dataset from: " + dataset_dir)

Loading dataset from: /content/CIFAKE


In [16]:
# Import all of the data using dataset from directory
# If there isn't enough RAM available, consider using Tensorflow Datasets

import tensorflow as tf

img_height = 32 # The dataset is all 32px but this is here just to make sure
img_width = 32
batch_size = 32

# Load the training data
train_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_dir + "/train",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Load the validation data
val_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_dir + "/test",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Quick sanity check to make sure it's all loaded properly
print("Training Classes:")
class_names = train_ds.class_names
print(class_names)

print("Testing Classes:")
class_names = train_ds.class_names
print(class_names)

Found 100000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Training Classes:
['FAKE', 'REAL']
Testing Classes:
['FAKE', 'REAL']


In [17]:
# Building the CNN
layers = []
layers.append(tf.keras.layers.Rescaling(1./255)) # Normalise pixel values
layers.append(tf.keras.layers.Conv2D(32, 3, activation='relu'))
layers.append(tf.keras.layers.MaxPooling2D())
layers.append(tf.keras.layers.Flatten())

# Building the ANN
layers.append(tf.keras.layers.Dense(64, activation='relu'))
layers.append(tf.keras.layers.Dense(1, activation='sigmoid'))

# Create and compile the model from layers
model = tf.keras.Sequential(layers)
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Build the model so we can see a summary
model.build(input_shape=(None, 32, 32, 3))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 32, 32, 3)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 7200)              0         
                                                                 
 dense_2 (Dense)             (None, 64)                460864    
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                      

In [18]:
# Check if GPUs are available for training
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [19]:
# Finally, train the model

print("Starting training...")
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5,
  verbose=1
)

!mkdir TrainedModelSave
model.save("/content/TrainedModelSave") # Saving the model

print("Training finished.")

Starting training...
Epoch 1/5
3125/3125 [==============================] - 89s 28ms/step - loss: 0.3273 - accuracy: 0.8595 - precision_1: 0.8509 - recall_1: 0.8719 - val_loss: 0.2918 - val_accuracy: 0.8804 - val_precision_1: 0.9086 - val_recall_1: 0.8459
Epoch 2/5
3125/3125 [==============================] - 84s 27ms/step - loss: 0.2259 - accuracy: 0.9097 - precision_1: 0.9034 - recall_1: 0.9175 - val_loss: 0.2397 - val_accuracy: 0.9069 - val_precision_1: 0.9448 - val_recall_1: 0.8643
Epoch 3/5
3125/3125 [==============================] - 88s 28ms/step - loss: 0.1889 - accuracy: 0.9258 - precision_1: 0.9226 - recall_1: 0.9295 - val_loss: 0.1905 - val_accuracy: 0.9250 - val_precision_1: 0.9218 - val_recall_1: 0.9287
Epoch 4/5
3125/3125 [==============================] - 89s 28ms/step - loss: 0.1643 - accuracy: 0.9355 - precision_1: 0.9324 - recall_1: 0.9392 - val_loss: 0.1920 - val_accuracy: 0.9251 - val_precision_1: 0.9358 - val_recall_1: 0.9128
Epoch 5/5
3125/3125 [==================